In [1]:
import sys

PWD = '../../..'
sys.path.append(PWD)

import numpy as np
import pandas as pd

import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
import keras_tuner as kt

from src.network_utils import count_params
from src.cross_validator import KerasTunerCrossValidator
from src.dataset import load_dataset_train_test, load_dataset_train_val
from src.models import mlp_builder as bare_model_builder

In [2]:
PROJECT_NAME = 'mlp'

PLANE = 2
CHANNEL = 11

OVERWRITE = False

DATASET_PATH = PWD + '/data/dataset/dataset.pkl'
TRIALS_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/tuner'
CROSSVAL_DIR = PWD + f'/data/model_selection/channel_{PLANE}_{CHANNEL}/cross_val'

LR = 0.01
ES_MIN_DELTA = 0.01

N_EPOCHS = 3000
BATCH_SIZE = 2048
MAX_TRIALS = 40
EXECUTIONS_PER_TRIAL = 2

TOP_N = 5
CROSSVAL_N_CV = 5
CROSSVAL_N_EXEC = 2
LOSS_WEIGHT = 1000

In [3]:
X_base_train, _, y_base_train, _ = load_dataset_train_test(PWD, PLANE, CHANNEL)
X_train, X_val, y_train, y_val = load_dataset_train_val(PWD, PLANE, CHANNEL)

X_base_train.shape, X_train.shape, X_val.shape

((22412, 24), (17929, 24), (4483, 24))

# Model

In [4]:
def model_builder(hp: kt.HyperParameters) -> keras.Model:
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=8, step=1, default=2)
    hp_units_mult = hp.Choice("units_mult", values=[1, 2, 4, 8, 16, 32], default=4)
    hp_unit_decrease_factor = hp.Choice("unit_decrease_factor", values=[1.0, 1.5, 2.0], default=1.5)
    hp_batch_normalization = hp.Boolean("batch_normalization", default=False)
    hp_input_batch_normalization = hp.Boolean("input_batch_normalization", default=False)
    hp_dropout = hp.Choice("dropout", values=[0.0, 0.2, 0.5])

    model = bare_model_builder(hp_n_hidden_layers, hp_units_mult, hp_unit_decrease_factor, hp_batch_normalization,
                               hp_input_batch_normalization, hp_dropout)
    model.compile(loss='mse', optimizer=optimizers.Adam(LR), loss_weights=LOSS_WEIGHT)
    return model

In [5]:
model_builder(kt.HyperParameters()).summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                300       
                                                                 
 dense_1 (Dense)             (None, 18)                234       
                                                                 
 dense_2 (Dense)             (None, 1)                 19        
                                                                 
Total params: 553
Trainable params: 553
Non-trainable params: 0
_________________________________________________________________


In [6]:
model_callbacks = [callbacks.EarlyStopping(patience=60, min_delta=ES_MIN_DELTA),
                   callbacks.ReduceLROnPlateau(monitor='loss', factor=0.9, patience=10)]

# Bayesian tuner

In [7]:
bayesian_tuner = kt.BayesianOptimization(model_builder, objective='val_loss', executions_per_trial=EXECUTIONS_PER_TRIAL,
                                         max_trials=MAX_TRIALS, directory=TRIALS_DIR, project_name=PROJECT_NAME,
                                         overwrite=OVERWRITE)

bayesian_tuner.search(X_train, y_train, validation_data=[X_val, y_val], epochs=N_EPOCHS, callbacks=model_callbacks,
                      batch_size=BATCH_SIZE, verbose=3)

# Results

In [8]:
for i, hyperparameters in enumerate(bayesian_tuner.get_best_hyperparameters(TOP_N)):
    print(f'========== Model {i} ==========')
    print(hyperparameters.get_config()['values'])
    model_tmp = model_builder(hyperparameters)
    print('Number of parameters:', count_params(model_tmp))

========== Model 0 ==========
{'n_hidden_layers': 2, 'units_mult': 32, 'unit_decrease_factor': 1.5, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5}
Number of parameters: 17041
========== Model 1 ==========
{'n_hidden_layers': 5, 'units_mult': 4, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 985
========== Model 2 ==========
{'n_hidden_layers': 7, 'units_mult': 1, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 1217
========== Model 3 ==========
{'n_hidden_layers': 4, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 100513
========== Model 4 ==========
{'n_hidden_layers': 7, 'units_mult': 2, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropo

# Cross-validation for top 5 models

In [9]:
cross_validator = KerasTunerCrossValidator(bayesian_tuner, X_base_train, y_base_train, model_builder,
                                           directory=CROSSVAL_DIR, project_name=PROJECT_NAME, n_epochs=N_EPOCHS,
                                           batch_size=BATCH_SIZE, n_top=TOP_N, es_min_delta=ES_MIN_DELTA,
                                           n_cv=CROSSVAL_N_CV, n_executions=CROSSVAL_N_EXEC, overwrite=OVERWRITE)
model_scores = cross_validator()

{'n_hidden_layers': 2, 'units_mult': 32, 'unit_decrease_factor': 1.5, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.5}
Number of parameters: 17041
Got score: 8.8288 (8.9076, 8.7500)
Got score: 8.5195 (8.5286, 8.5104)
Got score: 8.2369 (8.2880, 8.1858)
Got score: 8.5222 (8.5608, 8.4835)
Got score: 9.0975 (9.0781, 9.1168)


{'n_hidden_layers': 5, 'units_mult': 4, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 985
Got score: 8.7216 (8.6889, 8.7543)
Got score: 8.4772 (8.5127, 8.4416)
Got score: 8.2328 (8.2164, 8.2491)
Got score: 8.4582 (8.4531, 8.4633)
Got score: 8.9443 (9.0504, 8.8382)


{'n_hidden_layers': 7, 'units_mult': 1, 'unit_decrease_factor': 1.5, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 1217
Got score: 8.8654 (8.7519, 8.9790)
Got score: 8.5524 (8.5978, 8.5070)
Got score: 8.2392 (8.2610, 8.2173)
Got score: 8.4952 (8.5104, 8.4800)
Got score: 8.9934 (8.9019, 9.0850)


{'n_hidden_layers': 4, 'units_mult': 16, 'unit_decrease_factor': 2.0, 'batch_normalization': True, 'input_batch_normalization': True, 'dropout': 0.2}
Number of parameters: 100513
Got score: 9.3227 (9.0474, 9.5980)
Got score: 8.6714 (8.5404, 8.8024)
Got score: 8.4091 (8.4466, 8.3716)
Got score: 8.7006 (8.7315, 8.6697)
Got score: 9.3642 (9.2078, 9.5205)


{'n_hidden_layers': 7, 'units_mult': 2, 'unit_decrease_factor': 1.0, 'batch_normalization': False, 'input_batch_normalization': True, 'dropout': 0.0}
Number of parameters: 457
Got score: 8.7428 (8.7586, 8.7270)
Got score: 8.4237 (8.3761, 8.4712)
Got score: 8.1758 (8.1581, 8.1935)
Got score: 8.5151 (8.6252, 8.4050)
Got score: 8.8439 (8.8313, 8.8565)


In [10]:
mean_scores = [f"{np.mean(scores):0.2f}" for scores in model_scores.values()]
std_scores = [f"{np.std(scores):0.2f}" for scores in model_scores.values()]
n_params = [count_params(model_builder(hyperparameters)) for hyperparameters in
            bayesian_tuner.get_best_hyperparameters(TOP_N)]

df = pd.DataFrame({'mean': mean_scores, 'std': std_scores, 'n_params': n_params}, index=model_scores.keys())
df.index.name = 'Model'
df

,mean,std,n_params
Model,,,
0,8.64,0.30,17041
1,8.57,0.24,985
2,8.63,0.27,1217
3,8.89,0.38,100513
4,8.54,0.24,457
